### Day 17

### Part 1:
- probes are fired with a velocity (vx,vy) from position (x,y) = (0,0)
- Movement happens in steps. Each step is:
    - Increase x position by vx
    - Increase y position by vy
    - Decrease vx towards 0 by 1 due to drag
    - Decrease vy by 1 due to gravity
- Find the probe velocity that hits the target and achieves maximum height
   
Thoughts:
- Could work it out mathematically, but might need the trajectory. So simulate with a class for the probe, and another for the submarine firing them

In [1]:
class Probe(object):
    def __init__(self,velocity,target_x=[0,0],target_y=[0,0]):
        self.steps = 0
        self.p = [0,0]
        self.v = velocity
        
        self.target_x=target_x
        self.target_y=target_y
        
        self.trajectory = [self.p]
    
    def one_step(self):
        vx,vy = self.v 
        new_p = [self.p[0]+self.v[0], self.p[1]+self.v[1]]
        new_v = [
            vx + 1*(vx<0) - 1*(vx>0), # drag
            vy - 1 # gravity
        ]
        
        # Update and save
        self.trajectory.append(new_p)
        self.p = new_p
        self.v = new_v
        self.steps += 1
        
    def n_steps(self,n):
        for ix in range(n):
            self.one_step()
    
    def check_if_hits_target(self):
        """Run steps until it's clear it will miss the target"""
        
        # We know it will miss if y velocity is negative 
        # and it's too far below the target area
        while (self.v[1] > 0) or (self.p[1] > self.target_y[0]):
            
            self.one_step()
            
            # Check final position
            if (self.p[0] <= self.target_x[1]) and (self.p[0] >= self.target_x[0]):
                if (self.p[1] <= self.target_y[1]) and (self.p[1] >= self.target_y[0]):
                    return True
                
        return False
    
    def get_max_height(self):
        """Look through trajectory and find max height"""
        max_height = 0
        for s in self.trajectory:
            max_height = max(max_height,s[1])
        return max_height
            
        
    def print_trajectory(self):
        # Work out how big the grid needs to be
        mx = [0,0] # Min and max x position
        my = [0,0]
        for s in self.trajectory:
            mx = [min(mx[0],s[0]),max(mx[1],s[0])]
            my = [min(my[0],s[1]),max(my[1],s[1])]
            
        # Grid size (between mx[0] and mx[1])
        nx = mx[1]-mx[0]
        ny = my[1]-my[0]
        
        grid = [["." for x in range(nx+1)] for y in range(ny+1)]
        
        # Add the target area
        for y in range(self.target_y[0],self.target_y[1]+1):
            for x in range(self.target_x[0],self.target_x[1]+1):
                if y <= my[1] and y >=my[0] and x <= mx[1] and x >= mx[0]:
                    grid[y-my[0]][x-mx[0]] = "T"
        
        # Add the trajectory points
        for s in self.trajectory:
            if s[1] <= my[1] and s[1] >=my[0] and s[0] <= mx[1] and s[0] >= mx[0]:
                grid[s[1]-my[0]][s[0]-mx[0]] = "#"
            else:
                print("not in grid")
                
        grid[-my[0]][-mx[0]] = "S"
        
        # Print it
        for row in grid[::-1]:
            print("".join(row))

In [2]:
class Submarine(object):
    def __init__(self,target_x,target_y):
        self.target_x = target_x
        self.target_y = target_y
        
    def find_highest_trajectory_probe(self,vxs = [0,10],vys=[0,10]):
        """Try a grid of probe velocities and find the highest 
        trajectory that hits the target zone."""
        
        best_height = 0
        best_p = None
        all_hits = []
        
        for vx in range(vxs[0],vxs[1]+1):
            for vy in range(vys[0],vys[1]+1):
                p = Probe([vx,vy],self.target_x,self.target_y)
                if p.check_if_hits_target():
                    all_hits.append([vx,vy])
                    height = p.get_max_height()
                    if height > best_height:
                        best_height = height
                        best_p = p
        return best_p, best_height,all_hits

In [3]:
# Test inputs
# p = Probe([7,2],[20,30],[-10,-5]) # Hit
# p = Probe([6,3],[20,30],[-10,-5]) # Hit
# p = Probe([9,0],[20,30],[-10,-5]) # Hit
p = Probe([17,-4],[20,30],[-10,-5]) # No hit
p.check_if_hits_target()
p.print_trajectory()

S................................................
.................................................
.................................................
.................................................
.................#...............................
....................TTTTTTTTTTT..................
....................TTTTTTTTTTT..................
....................TTTTTTTTTTT..................
....................TTTTTTTTTTT..................
....................TTTTTTTTTTT..#...............
....................TTTTTTTTTTT..................
.................................................
.................................................
.................................................
.................................................
................................................#


In [4]:
# Test inputs for submarine
s = Submarine([20,30],[-10,-5])
best_p,max_height,all_hits = s.find_highest_trajectory_probe(vxs=[0,10],vys=[0,10])
print(max_height)

45


In [ ]:
# Puzzle input:
s = Submarine([34,67],[-215,-186])
best_p,max_height,all_hits = s.find_highest_trajectory_probe(vxs=[0,20],vys=[0,500])
print(max_height)

### Part 2:
- How many trajectories hit?
- Need to check between vx = [0,max_target_x] and vy = [min_target_y, ?]
- But ? is the max height initial velocity from the previous part
- Since vx and vy are related, could speed this up by not checking every combination
    - e.g. if vx is too low, don't check smaller vxs
    - But should be doable without that

In [ ]:
# Test inputs for submarine
s = Submarine([20,30],[-10,-5])
best_p,max_height,all_hits = s.find_highest_trajectory_probe(vxs=[0,30],vys=[-40,60])
print(max_height)
print(len(all_hits))

In [ ]:
# Puzzle input:
s = Submarine([34,67],[-215,-186])
best_p,max_height,all_hits = s.find_highest_trajectory_probe(vxs=[0,67],vys=[-215,214])
print(max_height)
print(len(all_hits))